In [ ]:
# NBVAL_IGNORE_OUTPUT
import lineapy
%load_ext lineapy

x = 100
y = 10

z = [x]
art = lineapy.save(z, "z")
pipeline = lineapy.create_workflow([art.name], framework='AIRFLOW', output_dir="~/airflow/dags")
pipeline_dir = pipeline.output_dir

In [ ]:
# NBVAL_IGNORE_OUTPUT
module_path = pipeline_dir/"z_module.py"
print(module_path.read_text())

In [ ]:
dag_path = pipeline_dir/"z_dag.py"
print(dag_path.read_text())

In [ ]:
# Cleanup pipeline files
import shutil
shutil.rmtree(pipeline_dir)